In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, roc_curve, auc

# --- 1. Data Loading and Exploration ---
data = pd.read_csv("WA_Fn-UseC_-Telco-Customer-Churn.csv")

print(data.head())
print(data.info())
print(data.describe())
print(data.isnull().sum())

sns.countplot(x='Churn', data=data)
plt.show()

# --- 2. Data Preprocessing ---
data['TotalCharges'] = pd.to_numeric(data['TotalCharges'], errors='coerce')
data['TotalCharges'].fillna(data['TotalCharges'].median(), inplace=True)

categorical_cols = data.select_dtypes(include=['object']).columns
label_encoder = LabelEncoder()
for col in categorical_cols:
    if col != 'customerID':
        data[col] = label_encoder.fit_transform(data[col])

# --- 3. Feature Engineering ---
data['Tenure_MonthlyCharges_Ratio'] = data['MonthlyCharges'] / (data['tenure'] + 1)

# --- 4. Model Training ---
X = data.drop(['Churn', 'customerID'], axis=1)
y = data['Churn']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

model = LogisticRegression(max_iter=1000)
model.fit(X_train, y_train)

# --- 5. Model Evaluation ---
y_pred = model.predict(X_test)

print("Accuracy:", accuracy_score(y_test, y_pred))
print("\nConfusion Matrix:\n", confusion_matrix(y_test, y_pred))
print("\nClassification Report:\n", classification_report(y_test, y_pred))

fpr, tpr, thresholds = roc_curve(y_test, model.predict_proba(X_test)[:,1])
roc_auc = auc(fpr, tpr)

plt.figure()
plt.plot(fpr, tpr, color='darkorange', lw=2, label='ROC curve (area = %0.2f)' % roc_auc)
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic')
plt.legend(loc="lower right")
plt.show()

# --- 6. Interpretation and Insights ---
feature_importances = pd.DataFrame({'feature': X.columns, 'importance': model.coef_[0]})
feature_importances = feature_importances.sort_values('importance', ascending=False)

print("\nFeature Importances:\n", feature_importances)

plt.figure(figsize=(10, 6))
sns.barplot(x='importance', y='feature', data=feature_importances)
plt.title('Feature Importances in Logistic Regression Model')
plt.show()